In [113]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import BatchNormalization

es = EarlyStopping(monitor = "val_loss", mode = "min", patience = 5)

EPOCHS = 1000

In [114]:
df = pd.read_csv('us_merged.csv', sep=",")

In [115]:
df

,Unnamed: 0,title,review,rating
0,0,Creepy and atmospheric,The Hollywood version of 'Dark Water' is based...,8/10
1,1,Teenagers!,"Don't know about you all, but I've sort of had...",9/10
2,2,Pretty good with a great performance by Connely,"Dark Water is another one of those slow PG-13,...",8/10
3,3,Dark Water Has Depth,Of all the recent remakes of Japanese horror f...,9/10
4,4,Good character-driven human drama; I blame the...,"Once more, a great injustice has been done by ...",7/10
...,...,...,...,...
3297,214,Uninspired,"I haven't seen that many scary movies, but one...",7/10
3298,215,Great progress for further Asian remakes! The ...,I came back from the 'The Uninvited' and I am ...,8/10
3299,216,The ending makes it filled with flaws and conf...,I found The Uninvited on IMDb and saw it got a...,6/10
3300,217,"A knock off of ""The Other"" not half bad but","A not bad, nifty thriller that unfortunately b...",5/10


In [116]:
df = df.drop('Unnamed: 0', axis=1)

In [117]:
df

,title,review,rating
0,Creepy and atmospheric,The Hollywood version of 'Dark Water' is based...,8/10
1,Teenagers!,"Don't know about you all, but I've sort of had...",9/10
2,Pretty good with a great performance by Connely,"Dark Water is another one of those slow PG-13,...",8/10
3,Dark Water Has Depth,Of all the recent remakes of Japanese horror f...,9/10
4,Good character-driven human drama; I blame the...,"Once more, a great injustice has been done by ...",7/10
...,...,...,...
3297,Uninspired,"I haven't seen that many scary movies, but one...",7/10
3298,Great progress for further Asian remakes! The ...,I came back from the 'The Uninvited' and I am ...,8/10
3299,The ending makes it filled with flaws and conf...,I found The Uninvited on IMDb and saw it got a...,6/10
3300,"A knock off of ""The Other"" not half bad but","A not bad, nifty thriller that unfortunately b...",5/10


In [118]:
df['review'][88]

'Oh boy.First of all I\'d like to say I\'m not against remakes in general. The remake has a very big disadvantage since it will doubtlessly be compared to the original (which has to be somewhat good, otherwise he wouldn\'t inspire the remake) so it\'s not really fair to be too hard on it. Besides, if you\'d loved the original it\'s always fun to look how other director/cast interpreted the thing.What I DO hate is when movie people start "improving" the original, putting in all sorts of their ideas and solutions in order to 1) distance themselves from the original and 2) make the movie more suited for intended audience. Perhaps some directors can pull this off, but usually (and especially when it is the case of American remakes) the "improving" consists of heavy overexplaining of the plot and augmented use of special effects. I don\'t get this - does Hollywood really think American audience is soo stupid they must be led by hand the entire movie, and God forbid something ambiguous was l

In [119]:
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import *
from tqdm import tqdm
from nltk import sent_tokenize,word_tokenize

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [120]:
main = []

# Storing all punctuations using RE library like !;,"% etc
re_puncs = re.compile('[%s]' % re.escape(string.punctuation))
# Storing all stop words like a, an, the, when, there, this etc
stop_word  = set(stopwords.words('english'))
stop_word.add("im")
# print(stop_word)


In [121]:
sentences_listed = [line.split(" ") for line in df['review']]

import gensim

word_model = gensim.models.Word2Vec(sentences = sentences_listed, vector_size = 100)
words = list(word_model.wv.key_to_index)
print(len(words))

8729


In [122]:
!pip install gensim

In [123]:
word_model.wv.most_similar("actress")

[('role', 0.9481761455535889),
 ('performance', 0.9480102062225342),
 ('Connelly', 0.9122159481048584),
 ('Jennifer', 0.8994792699813843),
 ('lead,', 0.8980265259742737),
 ('Gellar', 0.8972123861312866),
 ('Watts', 0.8940590620040894),
 ('actor', 0.8908982872962952),
 ('amazing', 0.8871170878410339),
 ('lead.', 0.881645143032074)]

In [124]:
word_model.wv.most_similar("death")

[('sister', 0.9220953583717346),
 ("mother's", 0.9195573329925537),
 ('house,', 0.9178259372711182),
 ('lives', 0.9161317944526672),
 ('niece', 0.9155774712562561),
 ('curse', 0.9150200486183167),
 ('family', 0.9147472381591797),
 ('father', 0.9142229557037354),
 ('woman', 0.9137316942214966),
 ('daughter', 0.9127646088600159)]

In [125]:
word_model.wv.most_similar("scary")

[('bad', 0.9063279628753662),
 ('scary,', 0.7712457776069641),
 ('good', 0.7582293748855591),
 ('truly', 0.7567077279090881),
 ('scary.', 0.7371953725814819),
 ('really', 0.7366823554039001),
 ('quite', 0.715095043182373),
 ('boring', 0.7149451375007629),
 ('Not', 0.704075813293457),
 ('fun', 0.6950154900550842)]

In [126]:
# Making Lemmatizing object
lem = WordNetLemmatizer()
# Using Porter Stemmer
p_stem = PorterStemmer()

# Traversing whole dataset
for i in tqdm(range(len(df['review']))):
    # Tokenization
    tokens = word_tokenize(str(df['review'][i]))
    # Converting all characters to lower case
    tokens = [w.lower() for w in tokens]
    # Remove all punctuations from sentenses
    tokens = [re_puncs.sub('', w) for w in tokens]
    # Checking all words is alphabets or not
    tokens = [i for i in tokens if i.isalpha()]
    # Removing all stop words from the sentenses
    tokens = [w for w in tokens if w not in stop_word]
    # Doing Lemmatizing of words
    tokens = [lem.lemmatize(w) for w in tokens]
    # Stemming process
    tokens = [p_stem.stem(w) for w in tokens]
    # Finally convert to string
    r = ' '.join(tokens)
    # Storing the final string into main list
    main.append(r)


100%|██████████| 3302/3302 [00:31<00:00, 103.54it/s]


In [127]:
df['translated'] = main

In [128]:
df

,title,review,rating,translated
0,Creepy and atmospheric,The Hollywood version of 'Dark Water' is based...,8/10,hollywood version dark water base japanes horr...
1,Teenagers!,"Don't know about you all, but I've sort of had...",9/10,nt know sort teenag walter sall dark water flo...
2,Pretty good with a great performance by Connely,"Dark Water is another one of those slow PG-13,...",8/10,dark water anoth one slow atmospher creepi gho...
3,Dark Water Has Depth,Of all the recent remakes of Japanese horror f...,9/10,recent remak japanes horror film say dark wate...
4,Good character-driven human drama; I blame the...,"Once more, a great injustice has been done by ...",7/10,great injustic done imdb voter dark water curr...
...,...,...,...,...
3297,Uninspired,"I haven't seen that many scary movies, but one...",7/10,nt seen mani scari movi one thing know sure mo...
3298,Great progress for further Asian remakes! The ...,I came back from the 'The Uninvited' and I am ...,8/10,came back uninvit surpris horror fanat everi h...
3299,The ending makes it filled with flaws and conf...,I found The Uninvited on IMDb and saw it got a...,6/10,found uninvit imdb saw got okay rate horrorsus...
3300,"A knock off of ""The Other"" not half bad but","A not bad, nifty thriller that unfortunately b...",5/10,bad nifti thriller unfortun break rule ca nt t...


In [129]:
#df['translated'][88]

In [130]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['translated'])

In [131]:
print(len(tokenizer.word_counts))

17869


In [132]:
counter = 0
for key, value in tokenizer.word_counts.items():
    if value > 2400:
        print(key, value)
        counter+=1

print(counter)

horror 3659
film 6132
nt 5906
ring 2443
good 2610
one 3182
watch 2558
like 3149
movi 9678
see 2647
10


**grupa 1** fright(ened), shocked/shocking, horror/horrified, horrendous, overwhelming, terror/terrified, disturbing, bloody, disgust(ing), brutal, gory/gore, barbarous, repulsed/-ing, gross(ed out)

In [133]:
count1 = 0
num=tokenizer.word_counts['fright']
print(num)
count1+=num

81


In [134]:
num=tokenizer.word_counts['frighten']
print(num)
count1+=num

308


In [135]:
num=tokenizer.word_counts['shock']
print(num)
count1+=num

263


In [136]:
num=tokenizer.word_counts['horrifi']
print(num)
count1+=num

94


In [137]:
num=tokenizer.word_counts['horrif']
print(num)
count1+=num

57


In [138]:
num=tokenizer.word_counts['horrend']
print(num)
count1+=num

12


In [139]:
num=tokenizer.word_counts['overwhelm']
print(num)
count1+=num

9


In [140]:
num=tokenizer.word_counts['overwhelming']
print(num)
count1+=num

1


In [141]:
num=tokenizer.word_counts['disturb']
print(num)
count1+=num

326


In [142]:
num=tokenizer.word_counts['terror']
print(num)
count1+=num

153


In [143]:
num=tokenizer.word_counts['blood']
print(num)
count1+=num

174


In [144]:
num=tokenizer.word_counts['terrifi']
print(num)
count1+=num

230


In [145]:
num=tokenizer.word_counts['disgust']
print(num)
count1+=num

25


In [146]:
num=tokenizer.word_counts['brutal']
print(num)
count1+=num

23


In [147]:
num=tokenizer.word_counts['gore']
print(num)
count1+=num

372


In [148]:
num=tokenizer.word_counts['repuls']
print(num)
count1+=num

14


In [149]:
num=tokenizer.word_counts['gross']
print(num)
count1+=num

28


In [150]:
print('WYNIK DLA GRUPY PIERWSZEJ TOOOOO',count1)

WYNIK DLA GRUPY PIERWSZEJ TOOOOO 2170


**grupa 2** afraid, alarmed, threatening, hair-rising, distress, helpless(ness), anxiety/anxious, worried, apprehension, uneasy, chilling, eerie, intimidating, unnerving, appalling

In [151]:
count2=0
num=tokenizer.word_counts['distress']
print(num)
count2+=num

13


In [152]:
num=tokenizer.word_counts['afraid']
print(num)
count2+=num

85


In [153]:
num=tokenizer.word_counts['alarm']
print(num)
count2+=num

8


In [154]:
num=tokenizer.word_counts['threaten']
print(num)
count2+=num

26


In [155]:
num=tokenizer.word_counts['anxious']
print(num)
count2+=num

2


In [156]:
num=tokenizer.word_counts['anxieti']
print(num)
count2+=num

19


In [157]:
num=tokenizer.word_counts['worri']
print(num)
count2+=num

43


In [158]:
num=tokenizer.word_counts['apprehens']
print(num)
count2+=num

KeyError: 'apprehens'

In [159]:
num=tokenizer.word_counts['uneasi']
print(num)
count2+=num

24


In [160]:
num=tokenizer.word_counts['chilli']
print(num)
count2+=num

4


In [161]:
num=tokenizer.word_counts['eeri']
print(num)
count2+=num

159


In [162]:
num=tokenizer.word_counts['helpless']
print(num)
count2+=num

9


In [163]:
num=tokenizer.word_counts['unnerv']
print(num)
count2+=num

46


In [164]:
num=tokenizer.word_counts['appal']
print(num)
count2+=num

6


In [165]:
num=tokenizer.word_counts['fear']
print(num)
count2+=num

280


In [166]:
print('WYNIK DLA GRUPY DRUGIEJ TOOOOO',count2)

WYNIK DLA GRUPY DRUGIEJ TOOOOO 724
